# Sire - quick demo

Import sire - we import it as `sr` as we are lazy ;-)

In [ ]:
import sire as sr

Load some molecules - these can be loaded from local files or from URLs!

Sire can read many different molecular file formats.

It will automatically detect the file format and load the molecules using the right parser.

In [ ]:
mols = sr.load("input.top", "input.rst")

In [ ]:
mols

# Interoperability of molecular information 1 - NGLView

The information contained in this system can be converted to a format recognised by NGLView.

This let's us easily view the molecules in 3D.

In [ ]:
mols.view()

That was a little slow, as converting all of the water molecules took time. We often don't want to see all of the water molecules. Sire has in-built search, so we can search for all molecules which are `not water`.

In [ ]:
mols_no_water = mols["not water"]

In [ ]:
mols_no_water.view()

That was much quicker!

NGLView has lots of options for controlling the way the molecules are viewed. This can be controlled via options to the `view()` function.

In [ ]:
mols_no_water.view(protein=["licorice", "surface:0.5"], default="spacefill")

What molecules are in this system? We can quickly get an idea by printing them.

In [ ]:
mols_no_water

The first molecule is the protein. The second looks like the ligand. The remainder are ions. Let's look at what appears to be the ligand.

In [ ]:
ligand = mols_no_water[1]
ligand.view()

# Interoperability of molecular information 2 - RDKit

sire can convert molecular information to and from RDKit. This means that we can use smiles and smarts strings to look for molecules. Maybe we know that the ligand contains an ether group. We could use that to search for the ligand by finding all non-protein matches for the smarts string `A-[#8]-A`.

In [ ]:
match = mols_no_water["not protein"]["smarts A-[#8]-A"]
match

There is one match, which will be in `match.group(0)`. We can get the molecule containing the match using the `.molecule()` function.

In [ ]:
ligand = match.group(0).molecule()
ligand.view()

Interoperability with RDKit let's sire render 2D views of molecules too!

In [ ]:
ligand.view2d()

And their smiles strings!

In [ ]:
smiles_string = ligand.smiles()
smiles_string

Indeed, we could have found the ligand via its smiles string...

In [ ]:
match = mols_no_water["not protein"]["smiles CCC(CC)OC1C=C(C(=O)[O-])CC([NH3+])C1NC(C)=O"]
match

In [ ]:
match.group(0).view()

This was possible because, under the hood, sire converts the molecule into a native RDKit molecule

In [ ]:
rdkit_mol = sr.convert.to(ligand, "rdkit")
print(type(rdkit_mol))
rdkit_mol

It can also go the other way - converting a native RDKit molecule into a sire molecule!

In [ ]:
sire_mol = sr.convert.to(rdkit_mol, "sire")
sire_mol.view()

# Loading from URLs

We can load molecules from URLs - this can be really convenient, as the files are downloaded for you automatically.

In [ ]:
mols = sr.load("https://sire.openbiosim.org/m/kigaki.gro", "https://sire.openbiosim.org/m/kigaki.top", silent=True)

In [ ]:
mols.view()

This is a small peptide chain in water. Let's view this with the water disabled.

In [ ]:
mols.view(water=None, protein="licorice")

Viewing the entire system in 2D will convert everything to RDKit. Each molecule is grouped by structure.

Here we can see that there was one peptide chain, 7 sodium ions, 14 chloride ions and 3599 water molecules.

In [ ]:
mols.view2d()

# Interoperability of molecular information 3 - OpenMM

sire can convert molecular information to and from OpenMM. Like for RDKit, this converts the sire molecules into native OpenMM molecules. You could use the standard OpenMM API to run simulations if you wanted.

In [ ]:
openmm_mols = sr.convert.to(mols, "openmm")
openmm_mols

To make things easier, sire provides high-level functions for running minimisation and dynamics.

In [ ]:
mols = mols.minimisation().run().commit()

For dynamics, it is convenient to create a dynamics object that will run the simulation with the specified timestep.

In [ ]:
d = mols.dynamics(timestep="4 fs")

We can then run blocks of MD simulation, e.g.

In [ ]:
d.run("5 ps", save_frequency="0.5 ps")

In [ ]:
d.run("2 ps", save_frequency="0.5 ps")

We extract the results by "committing" the simulation.

In [ ]:
mols = d.commit()

Viewing a trajectory will use NGLView's support for playing movies!

In [ ]:
mols.view(protein="licorice")

Sire has an in-built molecular mechanics engine, so we can recalculate energies of any views. Here is the total energy.

In [ ]:
mols.energy()

And here is the energy between the protein and water

In [ ]:
mols["protein"].energy(mols["water"])

We can calculate these across trajectories. The results are put into pandas data frames.

In [ ]:
df = mols.trajectory().energy()

In [ ]:
df

Sire added a `pretty_plot()` function to the dataframe to make it easier to create quick graphs.

In [ ]:
df.pretty_plot()

You can do this for any energy component across the trajectory.

In [ ]:
df = mols["protein"].trajectory().energy(mols["water"])

In [ ]:
df

In [ ]:
df.pretty_plot()

# Saving molecules and trajectories

You can save molecule and trajectory information to files. Sire can write to many popular molecular file formats. For example, here we will save to Amber PRM / RST format.

In [ ]:
f = sr.save(mols.trajectory(), "md", format=["PRM", "RST"])
f

The simulation is thus saved, ready to be loaded again for further analysis.

In [ ]:
mols = sr.load(f)
mols.view()